In [26]:
import pandas as pd 
import numpy as np 
from numpy.fft import fft
import matplotlib.pyplot as plt
import fastparquet
import seaborn as sns
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf
from scipy.stats import skew
from scipy.stats import linregress
from statsmodels.tsa.ar_model import AutoReg
from scipy.signal import find_peaks
from scipy.stats import entropy
import joblib


**Прочитаем тестовые данные**

In [2]:
data = pd.read_parquet('test.parquet')

In [3]:
data

,id,dates,values
0,6125,"[2016-01-01T00:00:00.000000000, 2016-02-01T00:...","[1.85, -0.04, 0.19, -0.45, -0.75, -0.95, -2.91..."
1,26781,"[2016-01-01T00:00:00.000000000, 2016-02-01T00:...","[-0.41, 0.39, -0.47, -0.9, -1.46, -0.51, 0.51,..."
2,13333,"[2016-06-01T00:00:00.000000000, 2016-07-01T00:...","[-0.29, -1.26, 0.17, -1.22, 0.45, -0.94, 0.16,..."
3,53218,"[2016-01-01T00:00:00.000000000, 2016-02-01T00:...","[-1.47, 1.55, -0.03, 0.57, -0.57, 0.6, 0.27, 1..."
4,84204,"[2016-01-01T00:00:00.000000000, 2016-02-01T00:...","[2.33, 1.39, -1.03, -2.64, 1.89, 1.77, 1.43, 1..."
...,...,...,...
19995,80341,"[2016-03-01T00:00:00.000000000, 2016-04-01T00:...","[3.01, -0.58, 1.55, 0.48, -0.35, 1.93, 3.86, 2..."
19996,5891,"[2016-01-01T00:00:00.000000000, 2016-02-01T00:...","[0.66, 1.3, 2.78, -0.25, -1.97, -0.55, -1.08, ..."
19997,29091,"[2017-01-01T00:00:00.000000000, 2017-02-01T00:...","[0.09, 0.44, 1.55, 0.15, 0.3, 0.19, 0.34, 1.05..."
19998,85877,"[2017-04-01T00:00:00.000000000, 2017-05-01T00:...","[0.28, 1.92, 1.14, 2.4, 1.46, 1.08, -0.12, 0.6..."


**В тестовых данных тоже есть пропуски, но пропуски и в тесте и в трейне такого характера, что значение поля `value` в случае пропуска - это массив из `None` вместо временного ряда.** Можно было бы заполнить эти пропуски на основании временного промежутка, когда они были записаны, но, как выяснилось при формировании признаков, время записи ряда не влияет на метку классса. Поэтому, дабы получить submission соответствующей размерности, я заменю пустые временные ряды случайными рядами из датасета. Во всяком случае пропусков всего лишь 21, что около 0.1% от тестовых данных. Это несущественно повлияет на качетсво прогноза.

In [4]:
def has_none(time_series):
    return any(item is np.inf or item is np.nan or item is None  for item in time_series)

none_id = data[data['values'].apply(lambda x: has_none(x))].index.tolist() + data[data['dates'].apply(lambda x: has_none(x))].index.tolist()

In [13]:
available_values = data.drop(none_id)['values']

for idx in none_id:
        random_value = np.random.choice(available_values)
        data.at[idx, 'values'] = random_value

**Получим статистические признаки из тестового датасета**

In [16]:
stat_feachure = pd.DataFrame()

In [17]:
def linear_trend(values):
    x = np.arange(len(values)) 
    slope, intercept, r_value, p_value, std_err = linregress(x, values)
    return slope  

def phase_duration(series):
    mean_value = np.mean(series)
    above_mean = series > mean_value
    max_phase = 1
    current_phase = 1

    for i in range(1, len(above_mean)):
        if above_mean[i] == above_mean[i - 1]:
            current_phase += 1
        else:
            max_phase = max(max_phase, current_phase)
            current_phase = 1

    return max(max_phase, current_phase)


def max_trend_length(series):
    diffs = np.diff(series)
    trend_length = 1
    max_trend = 1

    for i in range(1, len(diffs)):
        if diffs[i] * diffs[i-1] > 0:  # Проверка направления тренда
            trend_length += 1
        else:
            max_trend = max(max_trend, trend_length)
            trend_length = 1
    
    max_trend = max(max_trend, trend_length)  # На случай, если тренд до конца
    return max_trend

def time_series_entropy(series):
    value_counts = np.histogram(series, bins=10, density=True)[0]
    return entropy(value_counts + 1e-10)


def make_stat_feachures(stat_feachure):

    stat_feachure['max'] = data['values'].apply(lambda x: np.max(x))
    stat_feachure['min'] = data['values'].apply(lambda x: np.min(x))
    stat_feachure['median'] = data['values'].apply(lambda x: np.median(x))
    
    stat_feachure['mean'] = data['values'].apply(lambda x: np.mean(x))
    stat_feachure['var'] = data['values'].apply(lambda x: np.var(x))
    stat_feachure['amplitude'] = data['values'].apply(lambda x: np.max(x) - np.min(x))
    stat_feachure['peaks_valleys'] = data['values'].apply(lambda x: len(find_peaks(x)[0]) + len(find_peaks(-1*np.array(x))[0]))
    
    # stat_feachure['valleys'] = data['values'].apply(lambda x: len(find_peaks(-1*np.array(x))[0]))
    
    stat_feachure['skew'] = data['values'].apply(lambda x: skew(x))
    stat_feachure['percentile_25'] = data['values'].apply(lambda x: np.percentile(x, 25))
    stat_feachure['percentile_75'] = data['values'].apply(lambda x: np.percentile(x, 75))
    
    # stat_feachure['mean_abs_deviation'] = data['values'].apply(lambda x: np.mean(np.abs(x - np.mean(x))))
    
    stat_feachure['linear_trend'] = data['values'].apply(linear_trend)
    
    stat_feachure['mean_abs_diff'] = data['values'].apply( lambda x: np.mean(np.abs(np.diff(x))))
    stat_feachure['phase_dur'] = data['values'].apply(phase_duration)
    stat_feachure['max_trend_len'] = data['values'].apply(max_trend_length)
    stat_feachure['Shenon_entropy'] = data['values'].apply(time_series_entropy)
  
    
    

make_stat_feachures(stat_feachure)

In [18]:
stat_feachure.reset_index(drop= True, inplace= True)

**Добавим в признаковое пространство первые 7 коэффициентов Фурье (абсолютных значений)**

In [19]:
def fourier_coefficients(values, n=7):
    fft_vals = fft(values)
    return np.abs(fft_vals[:n])

In [20]:
f_coefs = data['values'].apply(fourier_coefficients)
fourier_df = pd.DataFrame(f_coefs.tolist(), columns=[f'fourier_coeff_{i+1}' for i in range(7)])

fourier_df

,fourier_coeff_1,fourier_coeff_2,fourier_coeff_3,fourier_coeff_4,fourier_coeff_5,fourier_coeff_6,fourier_coeff_7
0,2.160000,15.001185,10.514341,12.523787,20.219856,32.547124,13.125348
1,0.930000,14.076701,6.617842,6.356383,9.091009,16.041176,6.212912
2,2.176292,17.732854,8.149558,9.648646,11.542821,15.174816,6.176373
3,12.420000,21.115132,15.422313,17.586300,12.171062,10.387986,22.704324
4,6.155851,10.126668,14.170587,30.127301,5.645795,12.125380,6.375613
...,...,...,...,...,...,...,...
19995,1.821829,29.685297,7.569215,10.371833,10.953747,9.363467,16.827401
19996,5.940000,8.685339,2.366122,13.965024,7.406123,15.933842,9.781599
19997,3.780000,25.504289,23.635998,18.943334,5.666533,2.499803,10.928970
19998,1.710050,20.514132,11.755741,6.414518,5.788192,5.136502,4.784730


**Добавим в признаковое пространство первые 10 коэффициентов авторегрессии**

In [21]:
def ar_coefficients(values, lags=7):
    model = AutoReg(values, lags=lags).fit()
    return model.params[1:]  

In [22]:
%%time
ar_coefs = data['values'].apply(ar_coefficients)
ar_df = pd.DataFrame(ar_coefs.tolist(), columns=[f'ar_coef_{i+1}' for i in range(7)])

ar_df

CPU times: user 27 s, sys: 0 ns, total: 27 s
Wall time: 27 s


,ar_coef_1,ar_coef_2,ar_coef_3,ar_coef_4,ar_coef_5,ar_coef_6,ar_coef_7
0,0.107222,0.199202,0.100664,-0.054919,-0.268399,-0.208156,-0.094943
1,-0.000795,0.237289,-0.070995,-0.173122,0.038901,0.022556,-0.069356
2,0.384250,0.442362,-0.205726,-0.162302,0.216256,-0.185150,0.005972
3,0.222317,0.245295,0.093155,-0.183352,-0.051958,-0.036251,0.056937
4,0.122611,0.288118,0.449523,-0.124581,-0.193099,-0.186538,0.132546
...,...,...,...,...,...,...,...
19995,0.274222,0.097313,0.044403,0.022908,-0.117975,0.068892,0.109456
19996,0.436927,-0.190815,0.097675,-0.163571,-0.135573,-0.228875,-0.166924
19997,0.850089,-0.027343,-0.057979,0.083148,0.135297,-0.169521,-0.054614
19998,-0.597520,-0.348178,0.074889,0.359149,0.526973,0.413752,0.274122


**Объединим созданные признаки в один датафрейм**

In [31]:
feachures = pd.concat([stat_feachure, fourier_df, ar_df], axis = 1)


**Загрузим уже обученный StandartScaler и отмасштабируем тестовые данные**

In [32]:
scaler = joblib.load('standard_scaler.joblib')

In [33]:
feachures_scaled = pd.DataFrame(scaler.transform(feachures), columns= feachures.columns)
feachures_scaled

,max,min,median,mean,var,amplitude,peaks_valleys,skew,percentile_25,percentile_75,...,fourier_coeff_5,fourier_coeff_6,fourier_coeff_7,ar_coef_1,ar_coef_2,ar_coef_3,ar_coef_4,ar_coef_5,ar_coef_6,ar_coef_7
0,0.144235,-0.628634,0.429744,0.635838,2.273133,0.638960,-0.500925,-0.278362,-0.432540,1.001796,...,1.325585,2.513928,0.138895,-0.810922,0.002623,0.003901,-0.003837,-0.195039,-0.010305,-0.389760
1,1.014927,0.455972,0.088204,0.137142,0.201694,0.459507,-0.045823,0.613033,-0.432540,-0.752538,...,-0.348333,0.473382,-0.628159,-1.236177,0.004482,0.003136,-0.034270,0.061395,0.011914,-0.251566
2,0.543302,1.408014,-0.302128,-0.069055,-1.442445,-0.716904,-0.500925,0.711570,0.286794,-0.663936,...,0.020450,0.366278,-0.632214,0.279713,0.014490,0.002536,-0.031484,0.209393,-0.008089,0.155279
3,0.507024,-0.134536,1.015242,1.739956,0.678561,0.529295,1.228462,0.072811,0.613764,-0.025996,...,0.114946,-0.225493,1.201848,-0.357803,0.004873,0.003868,-0.036903,-0.014424,0.006250,0.430543
4,-0.532970,-0.050178,0.039412,-0.570779,0.155024,-0.397877,0.682340,-0.386759,-0.781308,0.647385,...,-0.866536,-0.010708,-0.610105,-0.750339,0.006963,0.005455,-0.021772,-0.132203,-0.008223,0.838905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1.027020,1.287503,-0.863231,0.088838,-0.974651,-0.218424,1.228462,1.315050,0.363087,-0.965185,...,-0.068154,-0.352149,0.549702,-0.153460,-0.002349,0.003650,0.016200,-0.069514,0.016376,0.714195
19996,-0.279018,-0.459917,1.747114,0.950180,1.222710,0.150450,-0.045823,-0.656126,-0.694116,1.887823,...,-0.601761,0.460113,-0.232152,0.487097,-0.016411,0.003888,-0.031810,-0.084199,-0.012301,-0.778527
19997,-1.331104,-1.255295,1.893489,0.916404,-0.337466,-0.058911,-0.773986,-1.716358,2.684575,0.151210,...,-0.863417,-1.200669,-0.104831,2.113682,-0.008433,0.003194,0.031710,0.141835,-0.006584,-0.171942
19998,-0.738550,-0.220912,-0.399711,-0.047271,-0.310152,-0.426116,-0.865007,-0.220541,-0.470714,0.744848,...,-0.845118,-0.874707,-0.786641,-3.585435,-0.024091,0.003786,0.102769,0.468678,0.049589,1.603551


**Теперь загрузим модель и получим предсказания на тестовом датасете**

In [35]:
model = joblib.load('random_forest_model.joblib')

In [50]:
test_prediction = model.predict_proba(feachures_scaled)[:, 1]

In [51]:
test_prediction

array([0.20446302, 0.22191615, 0.38605261, ..., 0.20676038, 0.40195305,
       0.18553924])

In [55]:
make_submission_file = pd.DataFrame(data['id'], columns= ['id'])
make_submission_file['score'] = test_prediction

In [56]:
make_submission_file

,id,score
0,6125,0.204463
1,26781,0.221916
2,13333,0.386053
3,53218,0.084246
4,84204,0.625991
...,...,...
19995,80341,0.359714
19996,5891,0.072651
19997,29091,0.206760
19998,85877,0.401953


**Запишем предсказания в файл `submission.csv`**

In [60]:
make_submission_file.to_csv('submission.csv', index= False)